# Multiple Linear Regression

Multiple linear regression on space with more than 3 dimensions.

In [1]:
# Dependencies

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

## Data Sourcing

In [2]:
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

# For simplicity, we will do linear regression with less data
# X = X[["MedInc", "HouseAge", "AveRooms"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=40, shuffle=True)

print(pd.concat([X_train, y_train], axis=1).head())

       MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
10346  6.0471      17.0  6.535849   0.979245      1713.0  3.232075     33.61   
1098   2.3833      24.0  5.643382   1.025735       726.0  2.669118     39.59   
2912   3.3261      16.0  4.699687   1.093848      2261.0  2.357664     35.36   
5576   4.7222      36.0  5.712000   1.072000       825.0  3.300000     33.84   
18435  5.5849      15.0  5.885662   1.000000      1752.0  3.179673     37.26   

       Longitude  MedHouseVal  
10346    -117.66        2.484  
1098     -121.74        0.951  
2912     -119.05        1.184  
5576     -118.30        2.396  
18435    -121.83        2.574  


## Linear Regression And Linear Algebra

In [3]:
# Transpose of X
X_np = X_train.to_numpy()
X_np = np.hstack([np.ones((len(X_np), 1)), X_np])
XT_np = np.matrix.transpose(X_np)

# Multiplication of XT and X
XT_X_np = np.linalg.matmul(XT_np, X_np)

# Inverse of multiplication of XT and X
XT_X_inv_np = np.linalg.inv(XT_X_np)

# Multiplication of XT and y
y_np = y_train.to_numpy()
XT_y_np = np.matmul(XT_np, y_np)

# B
B_np = np.matmul(XT_X_inv_np, XT_y_np)

# print(f"XT: {XT_np}")
# print(f"XT_X: {XT_X_np}")
# print(f"XT_X_inv: {XT_X_inv_np}")
# print(f"XT: {XT_np}")
print(f"B: {B_np}")

B: [-3.68810647e+01  4.36740621e-01  9.44020128e-03 -1.07322656e-01
  6.44881442e-01 -4.18695740e-06 -3.78496421e-03 -4.20705910e-01
 -4.33825568e-01]


## $TSS$, $RSS$, $MSE$, and $R^2$

In [4]:
def predict(X):
    featureCount = len(X) + 1
    pred_y = 0

    for i in range(featureCount):
        if i == 0:
            pred_y += B_np[i]
        else:
            pred_y += B_np[i]*X.iloc[i-1]

    return pred_y

y_pred = X_test.apply(predict, axis=1)
print(y_pred)

932      2.823049
14733    2.505164
16966    2.567401
9138     2.773571
16760    2.659205
11646    2.655326
9631     1.378955
1514     1.818290
7118     2.088042
5321     3.599308
17295    4.495395
2076     0.807418
2356     1.735609
15334    1.544897
4344     1.593150
731      2.549434
16013    3.362265
1463     2.356671
5813     1.679633
13892    1.535738
18497    2.030095
1947     1.156172
17016    2.611760
19584    2.430102
15206    2.028408
14083    2.487888
9619     2.173163
2300     1.441234
2933     0.868929
15463    1.925126
16913    6.173496
2124     1.553883
8010     2.665700
8800     4.573179
635      2.193071
1822     3.200510
2458     1.498494
5964     3.208933
2209     1.431840
13841    1.020210
dtype: float64


In [5]:
# TSS
tss = 0
for v in y_test:
    tss = tss + (v - y_test.mean())**2

# RSS
rss = 0
for (_, v_test), (_, v_pred) in zip(y_test.items(), y_pred.items()):
    rss = rss + (v_test - v_pred)**2

# R^2
r_squared = 1 - (rss/tss)

# MSE
mse = rss / len(y_test)

print(f"TSS: {tss}")
print(f"RSS: {rss}")
print(f"R^2: {r_squared}")
print(f"R^2 (sklearn.metrics): {r2_score(y_test, y_pred)}")
print(f"MSE: {mse}")
print(f"MSE (sklearn.metrics): {mean_squared_error(y_test, y_pred)}")

TSS: 69.99669182135997
RSS: 16.801662609656997
R^2: 0.7599649044481006
R^2 (sklearn.metrics): 0.7599649044481006
MSE: 0.42004156524142494
MSE (sklearn.metrics): 0.42004156524142494


## Comparing with scikit-learn's LinearRegression()

In [6]:
regressor = LinearRegression(fit_intercept=True).fit(X_train, y_train)
y_pred_sklearn = regressor.predict(X_test)

print(f"Features count: {regressor.n_features_in_}")
print(f"B: {regressor.intercept_, regressor.coef_}")
print(f"R^2 (sklearn.linear_model.LinearRegression): {r2_score(y_test, y_pred_sklearn)}")
print(f"MSE (sklearn.linear_model.LinearRegression): {mean_squared_error(y_test, y_pred_sklearn)}")

Features count: 8
B: (np.float64(-36.881064678873436), array([ 4.36740621e-01,  9.44020128e-03, -1.07322656e-01,  6.44881442e-01,
       -4.18695740e-06, -3.78496421e-03, -4.20705910e-01, -4.33825569e-01]))
R^2 (sklearn.linear_model.LinearRegression): 0.7599649044489143
MSE (sklearn.linear_model.LinearRegression): 0.420041565240001


## Optimizing By Omitting Less Impactful Features

### Optimize using p-value (statsmodel.api)

In [7]:
X_train_with_const = sm.add_constant(X_train)
ols_model = sm.OLS(y_train, X_train_with_const).fit()
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:            MedHouseVal   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.606
Method:                 Least Squares   F-statistic:                     3956.
Date:                Mon, 30 Dec 2024   Prob (F-statistic):               0.00
Time:                        23:08:44   Log-Likelihood:                -22584.
No. Observations:               20600   AIC:                         4.519e+04
Df Residuals:                   20591   BIC:                         4.526e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -36.8811      0.660    -55.909      0.0

### Optimize using RFE (skelarn.feature_selection)

In [16]:
rfe = RFE(estimator=regressor, n_features_to_select=3)
rfe.fit(X_train, y_train)

# Rank features
featureRanking = pd.DataFrame({
    'Feature': X.columns,
    'Ranking': rfe.ranking_,
    'Selected': rfe.get_support(),
})
print(featureRanking)

# Compare RFE with model without RFE
y_pred_rfe = rfe.predict(X_test)
print(f"R^2 (sklearn.feature_selection.RFE): {r2_score(y_test, y_pred_rfe)}")
print(f"MSE (sklearn.feature_selection.RFE): {mean_squared_error(y_test, y_pred_rfe)}")

      Feature  Ranking  Selected
0      MedInc        1      True
1    HouseAge        4     False
2    AveRooms        3     False
3   AveBedrms        2     False
4  Population        6     False
5    AveOccup        5     False
6    Latitude        1      True
7   Longitude        1      True
R^2 (sklearn.feature_selection.RFE): 0.7534539421856962
MSE (sklearn.feature_selection.RFE): 0.4314352107149757
